In [5]:
from collections import defaultdict
import gzip

iids = []
with open('iid.list') as f:
    for line in f:
        iid = line.strip()
        iids.append(iid)

mapAllele = defaultdict(set)

for iid in iids:
    with gzip.open(f'./genotypes/{iid}.chr3.1240k.vcf.gz', 'rt') as f:
        for line in f:
            if line.startswith('#'):
                continue
            _, bp, _, ref, alt, _, _, _, _, record = line.strip().split()
            bp = int(bp)
            GT = record.split(':')[0]
            gt = int(GT[0]) + int(GT[2])
            mapAllele[bp].add(gt)

from collections import Counter
c = Counter([len(v) for _, v in mapAllele.items()])
print(c)


Counter({3: 52629, 2: 15700, 1: 9323})


In [8]:
# impute accu map
path2sites="/mnt/archgen/users/hringbauer/data/GRG/vcf/pos/chr3.sites"
iids = []
with open('/mnt/archgen/users/yilei/IBDsim/BAM/1240k/iid.list') as f:
    for line in f:
        iid = line.strip()
        iids.append(iid)

import pickle

for cov in ['cov2', 'cov1', 'cov3over4', 'cov1over2', 'cov1over4', 'cov1over10']:
    count = 0
    accuMap = {}
    with open(path2sites) as f:
        for line in f:
            _, bp = line.strip().split()
            bp = int(bp)
            accuMap[bp] ={0:[], 1:[], 2:[]}

    for iid in iids:
        # first, determine groundtruth genotype at each site
        genotypes = {}
        with gzip.open(f'./genotypes/{iid}.chr3.1240k.vcf.gz', 'rt') as f:
            for line in f:
                if line.startswith('#'):
                    continue
                _, bp, _, ref, alt, _, _, _, _, record = line.strip().split()
                bp = int(bp)
                GT, _ = record.split(':')
                gt = int(GT[0]) + int(GT[2])
                genotypes[bp] = gt
        with open(f'./downsample/{cov}/{iid}/ch3/GLIMPSE_ligated/{iid}.chr3.1240k.vcf') as f:
            for line in f:
                if line.startswith('#'):
                    continue
                _, bp, _, ref, alt, _, _, _, _, record = line.strip().split()
                bp = int(bp)
                GT, _, GP, _ = record.split(':')
                gt = int(GT[0]) + int(GT[2])
                if gt != genotypes[bp]:
                    count += 1
                    #     print(f'inconsistent imputed genotype at {bp}')
                gp0, gp1, gp2 = GP.split(',')
                gp0, gp1, gp2 = float(gp0), float(gp1), float(gp2)
                accuMap[bp][genotypes[bp]].append((gp0, gp1, gp2))
            
    pickle.dump(accuMap, open(f'imputeAccuMap.{cov}', 'wb'))
    print(f'{count} GT switches for {cov}')    

12253 GT switches for cov2
19756 GT switches for cov1
25101 GT switches for cov3over4
36640 GT switches for cov1over2
75966 GT switches for cov1over4
205013 GT switches for cov1over10


In [ ]:
import pickle

accuMap = pickle.load(open('imputeAccuMap.cov1over2', 'rb'))

accuMap[2232424]